In [1]:
try:
    import elasticsearch
    from elasticsearch import Elasticsearch
    
    import os
    import sys
    
    import pandas as pd
    import json
    from ast import literal_eval
    from tqdm import tqdm
    import datetime
    
    import numpy as np
    from elasticsearch import helpers
    
    print("All Modules Loaded!")

except exception as e:
    print("some modules are missing{}".format(e))

All Modules Loaded!


In [2]:
for x in os.listdir():
    print(x)

.ipynb_checkpoints
ELK.ipynb
netflix_titles.csv
test.ipynb


In [3]:
df = pd.read_csv("netflix_titles.csv")

In [4]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...


In [5]:
df.columns

Index(['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description'],
      dtype='object')

In [6]:
df.shape

(7787, 12)

In [7]:
df["show_id"].unique().shape

(7787,)

In [8]:
#connect with elasticsearch : creating es instance

endpoint = "http://localhost:9200/"
es = Elasticsearch(timeout= 600,hosts = endpoint)

In [9]:
#Checking connection

es.ping()

True

In [10]:
df.isna().sum()

show_id            0
type               0
title              0
director        2389
cast             718
country          507
date_added        10
release_year       0
rating             7
duration           0
listed_in          0
description        0
dtype: int64

In [11]:
df= df.dropna()

In [12]:
df.shape

(4808, 12)

In [24]:
df["duration"].isnull().sum()

0

# We need to convert the data into APP format that elasticsearch can read

In [13]:
#converting to json 

#records - each row becomes a dictionary where key is column name and value is the data in the cell

df2 = df.to_dict("records")

In [14]:
df2[0]

{'show_id': 's2',
 'type': 'Movie',
 'title': '7:19',
 'director': 'Jorge Michel Grau',
 'cast': 'Demián Bichir, Héctor Bonilla, Oscar Serrano, Azalia Ortiz, Octavio Michel, Carmen Beato',
 'country': 'Mexico',
 'date_added': 'December 23, 2016',
 'release_year': 2016,
 'rating': 'TV-MA',
 'duration': '93 min',
 'listed_in': 'Dramas, International Movies',
 'description': 'After a devastating earthquake hits Mexico City, trapped survivors from all walks of life wait to be rescued while trying desperately to stay alive.'}

In [15]:
len(df2)

4808

# We need to convert data into ELK Format

In [16]:
def generator(df2):                              #generator : like a normal function
    for c, line in enumerate(df2):
        yield {                                   #yield used instead of return    
        "_index" : "netflix_shows",                  #gving index name for elk
        "_type" : "_doc",
        "_id" : line.get("show_id",None),              #giving id by show_id column of db
        "_source":{                                    #difining json data of our db
            "title":line.get("title",""),
            "description" : line.get("description",""),
            "duration" : line.get("duration",None),
            "director" : line.get("director",None),
            "rating" : line.get("rating",None),
            "country" : line.get("country",None)
            }}
    raise StopIteration 
    
    


In [17]:
mycustom = generator(df2)

In [18]:
mycustom

<generator object generator at 0x0000027A7C33AD60>

In [19]:
next(mycustom)    #The next() function returns the next item in an iterator.

{'_index': 'netflix_shows',
 '_type': '_doc',
 '_id': 's2',
 '_source': {'title': '7:19',
  'description': 'After a devastating earthquake hits Mexico City, trapped survivors from all walks of life wait to be rescued while trying desperately to stay alive.',
  'duration': '93 min',
  'director': 'Jorge Michel Grau',
  'rating': 'TV-MA',
  'country': 'Mexico'}}

# Upload data into Elastic Search

In [20]:
try:
    res = helpers.bulk (es,generator(df2))
    print("Working");
except Exception as e:
    pass

c:\python39\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchWarning)


In [21]:
netflix_shows

NameError: name 'netflix_shows' is not defined